In [ ]:
!pip install --quiet gradio opencv-python moviepy numpy mediapipe Pillow


In [ ]:
import os
import cv2
import numpy as np
import gradio as gr
import mediapipe as mp
from PIL import Image, ImageOps
from moviepy.editor import ImageClip, concatenate_videoclips, AudioFileClip
import random

mp_face_mesh = mp.solutions.face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def align_face(image_rgb, output_size=(1280,720), eye_target=(640,300)):
    h, w, _ = image_rgb.shape
    img = image_rgb.astype(np.uint8)
    res = mp_face_mesh.process(img)
    left_eye = None
    right_eye = None
    if res and res.multi_face_landmarks:
        lm = res.multi_face_landmarks[0].landmark
        left_idxs = [33, 133]
        right_idxs = [362, 263]
        left_eye = np.mean([(lm[i].x * w, lm[i].y * h) for i in left_idxs], axis=0)
        right_eye = np.mean([(lm[i].x * w, lm[i].y * h) for i in right_idxs], axis=0)
    if left_eye is None or right_eye is None:
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 5)
        if len(faces) == 0:
            return None
        x, y, wb, hb = max(faces, key=lambda f: f[2]*f[3])
        cx = x + wb/2
        cy = y + hb/3.0
        left_eye = np.array([cx - wb*0.18, cy])
        right_eye = np.array([cx + wb*0.18, cy])
    left_eye = np.array(left_eye, dtype=np.float32)
    right_eye = np.array(right_eye, dtype=np.float32)
    eyes_center = (left_eye + right_eye) / 2.0
    dx, dy = right_eye[0] - left_eye[0], right_eye[1] - left_eye[1]
    current_eye_dist = np.hypot(dx, dy)
    if current_eye_dist < 1.0:
        return None
    angle = np.degrees(np.arctan2(dy, dx))
    scale = 1.0
    cx, cy = eyes_center
    M = cv2.getRotationMatrix2D((cx, cy), angle, scale)
    eyes_center_rot = np.dot(M, np.array([cx, cy, 1.0]))
    tx, ty = eye_target[0] - eyes_center_rot[0], eye_target[1] - eyes_center_rot[1]
    M[0,2] += tx
    M[1,2] += ty
    out_w, out_h = output_size
    warped = cv2.warpAffine(img, M, (out_w, out_h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)
    return warped

def add_border_and_tilt(img_np):
    pil = Image.fromarray(img_np)
    bordered = ImageOps.expand(pil, border=30, fill='white')
    rot_angle = random.uniform(-5,5)
    return np.array(bordered.rotate(rot_angle, expand=True, fillcolor='white'))

def create_face_movie(files, music=None, duration_per_face=3.0, fade_duration=1.5):
    os.makedirs('outputs', exist_ok=True)
    clips = []
    for f in files:
        try:
            pil = Image.open(f.name).convert('RGB')
            img = np.array(pil)
        except:
            continue
        aligned = align_face(img)
        if aligned is None:
            continue
        stacked = add_border_and_tilt(aligned)
        clip = ImageClip(stacked).set_duration(duration_per_face)
        clips.append(clip)
    if not clips:
        return "Aucune image utilisable", None, None
    final = clips[0]
    for next_clip in clips[1:]:
        final = concatenate_videoclips([final.crossfadeout(fade_duration), next_clip.crossfadein(fade_duration)], method='compose')
    if music is not None:
        try:
            audio = AudioFileClip(music.name).set_duration(final.duration)
            final = final.set_audio(audio)
        except:
            pass
    out_path = 'outputs/face_movie_stack.mov'
    final.write_videofile(out_path, fps=24, codec='libx264', audio_codec='aac', ffmpeg_params=['-profile:v', 'baseline', '-pix_fmt', 'yuv420p'])
    return f"Vidéo générée avec {len(clips)} images", out_path, out_path

iface = gr.Interface(
    fn=create_face_movie,
    inputs=[
        gr.File(label="Images (multiple)", file_types=["image"], file_count="multiple"),
        gr.File(label="Musique (optionnel)", file_types=["audio"])
    ],
    outputs=[
        gr.Textbox(label="Statut", lines=5),
        gr.Video(label="Vidéo"),
        gr.File(label="Télécharger")
    ],
    title="Face Movie Stack — 16:9 + bordures + empilement",
    description="Photos alignées sur les yeux, format 16:9, empilement avec bordure blanche et fondu entre images"
)
iface.launch(share=True)
